In [41]:
# Import packages

import pandas as pd 
import numpy as np 
import os
from bs4 import BeautifulSoup
import time
import requests
import nfl_data_py as nfl
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

# Set certain settings for the notebook
pd.set_option("display.max_columns", None)

In [42]:
# Check current directory
os.getcwd()

'C:\\Users\\imacd_0odruq3\\Documents\\sports_gambling\\sports_gambling_algorithm'

In [43]:
# Read in data

elo_qb_data = pd.read_csv('../data/elo_qb_data.csv')
nfl_elo_raw = pd.read_csv('../data/nfl_elo_pred_1_output.csv')
nfl_per_game_raw = pd.read_csv('../data/nfl_per_game_pred_1_output.csv')
current_week_data = pd.read_csv('../data/current_week_data.csv')

In [44]:
# Append current week matchups to the nfl per game data 

current_week_data_nfl_append_home = current_week_data[['season', 'week', 'home', 'away', 
                                                       'home_full_name', 'away_full_name']]
current_week_data_nfl_append_home.columns = ['season', 'week', 'team', 'opponent',
                                            'team_full_name', 'opponent_full_name']
current_week_data_nfl_append_away = current_week_data[['season', 'week', 'away', 'home', 
                                                       'away_full_name', 'home_full_name']]
current_week_data_nfl_append_away.columns = ['season', 'week', 'team', 'opponent',
                                            'team_full_name', 'opponent_full_name']
nfl_per_game = pd.concat([nfl_per_game_raw, current_week_data_nfl_append_home, current_week_data_nfl_append_away])
nfl_per_game = nfl_per_game.reset_index()

In [45]:
# Append current week matchups to the nfl elo data 

current_week_data_nfl_append_elo = current_week_data[['season', 'week', 'home', 'away', 
                                                       'home_full_name', 'away_full_name',
                                                     'home_qb', 'away_qb']]
current_week_data_nfl_append_elo.columns = ['season', 'week', 'home', 'away',
                                            'home_full_name', 'away_full_name', 
                                           'qb1', 'qb2']
nfl_elo = pd.concat([nfl_elo_raw, current_week_data_nfl_append_elo])

In [46]:
# Function to get important passing/qb related stats

def get_passing_stats(nfl, elo):
    passing = nfl[['season', 'week', 'team_full_name', 'total_pass_attempts', 'total_passing_yards', 'completions', 'passing_epa',
                  'pass_tds', 'avg_cpoe', 'qb_epa', 'sacks_allowed', 'interceptions_thrown']]
    home_qb_elos = elo[['season', 'week', 'home_full_name', 'qb1', 'qb1_value_pre', 'qb1_adj']]
    home_qb_elos.columns = ['season', 'week', 'team_full_name', 'qb', 'qb_value_pre', 'qb_adj']
    away_qb_elos = elo[['season', 'week', 'away_full_name', 'qb2', 'qb2_value_pre', 'qb2_adj']]
    away_qb_elos.columns = ['season', 'week', 'team_full_name', 'qb', 'qb_value_pre', 'qb_adj']
    qb_elos = pd.concat([home_qb_elos, away_qb_elos])
    passing = passing.merge(qb_elos, how='inner', 
                           left_on = ['season', 'week', 'team_full_name'], 
                           right_on = ['season', 'week', 'team_full_name'])
    
    prev_games_qb_rolling = passing[['total_pass_attempts', 'total_passing_yards', 'completions', 'passing_epa',
                                    'pass_tds', 'avg_cpoe', 'qb_epa', 'sacks_allowed', 'interceptions_thrown',
                                    'qb_value_pre', 'qb_adj', 'qb']].groupby(by=['qb']).rolling(6, min_periods=1, closed='left').mean()
    prev_games_qb_rolling.columns = prev_games_qb_rolling.columns + '_last_6_avg'
    prev_games_qb_rolling = passing[['season', 'week', 'team_full_name']].join(prev_games_qb_rolling.reset_index(level='qb'))
    
    return passing, prev_games_qb_rolling

In [47]:
# Function to get important rushing stats

def get_rushing_stats(nfl):
    rushing = nfl[['season', 'week', 'team_full_name', 'total_rushes', 'total_rush_yards', 'rushing_epa', 'rush_tds', 'fumbles', 
                  'lost_fumbles']]
    prev_games_rushing = rushing[['team_full_name', 'total_rushes', 'total_rush_yards', 'rushing_epa', 'rush_tds', 'fumbles', 
                  'lost_fumbles']]
    prev_games_rushing['ypc'] = prev_games_rushing.apply(lambda x: x.total_rush_yards/x.total_rushes, axis=1)
    rushing_rolling = prev_games_rushing.groupby(by=['team_full_name']).rolling(6, min_periods=1, closed='left').mean()
    rushing_rolling.columns = rushing_rolling.columns + '_last_6_avg'
    rushing_rolling = rushing[['season', 'week']].join(rushing_rolling.reset_index(level='team_full_name'))
    
    return rushing_rolling

In [48]:
# Function to get important defensive stats

def get_defense_and_st_stats(nfl):
    defense = nfl[['season', 'week', 'team_full_name', 'total_rushes_allowed', 'total_rush_yards_allowed', 'rush_tds_allowed',
                  'total_pass_attempts_allowed', 'passing_yards_allowed', 'completions_allowed', 
                  'pass_tds_allowed', 'avg_cpoe_allowed', 'sacks', 'fumbles_forced', 'interceptions', 'fumbles_recovered',
                  'special_teams_epa', 'passing_epa_def', 'rushing_epa_def']]
    prev_games_defense = defense[['team_full_name', 'total_rushes_allowed', 'total_rush_yards_allowed', 'rush_tds_allowed',
                  'total_pass_attempts_allowed', 'passing_yards_allowed', 'completions_allowed', 
                  'pass_tds_allowed', 'avg_cpoe_allowed', 'sacks', 'fumbles_forced', 'interceptions', 'fumbles_recovered',
                  'special_teams_epa', 'passing_epa_def', 'rushing_epa_def']]
    defense_rolling = prev_games_defense.groupby(by=['team_full_name']).rolling(6, min_periods=1, closed='left').mean()
    defense_rolling.columns = defense_rolling.columns + '_last_6_avg'
    defense_rolling = defense[['season', 'week']].join(defense_rolling.reset_index(level='team_full_name'))
    
    return defense_rolling

In [49]:
# Get data frame with predictive stats

passing, qb_rolling = get_passing_stats(nfl_per_game, nfl_elo)
rushing_rolling = get_rushing_stats(nfl_per_game)
defense_st_rolling = get_defense_and_st_stats(nfl_per_game)

final_stats_df = qb_rolling.merge(
    rushing_rolling, left_on = ['season', 'week', 'team_full_name'], right_on = ['season', 'week', 'team_full_name']).merge(
    defense_st_rolling, left_on = ['season', 'week', 'team_full_name'], right_on = ['season', 'week', 'team_full_name'])
target_df = nfl_elo[['season', 'week', 'home', 'away', 'home_full_name', 'away_full_name']]
home_final_stats_df = final_stats_df.add_suffix('_home')
away_final_stats_df = final_stats_df.add_suffix('_away')
final_nfl = target_df.merge(home_final_stats_df, 
                                left_on = ['season', 'week', 'home_full_name'],
                                right_on = ['season_home', 'week_home', 'team_full_name_home'],
                                suffixes = [None, '_home'])
final_nfl = final_nfl.merge(away_final_stats_df, 
                                     left_on = ['season', 'week', 'away_full_name'],
                                     right_on = ['season_away', 'week_away', 'team_full_name_away'],
                                     suffixes = [None, '_away'])
final_nfl = final_nfl.dropna()

final_nfl

<ipython-input-47-ffce014894a1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_games_rushing['ypc'] = prev_games_rushing.apply(lambda x: x.total_rush_yards/x.total_rushes, axis=1)


,season,week,home,away,home_full_name,away_full_name,season_home,week_home,team_full_name_home,qb_home,total_pass_attempts_last_6_avg_home,total_passing_yards_last_6_avg_home,completions_last_6_avg_home,passing_epa_last_6_avg_home,pass_tds_last_6_avg_home,avg_cpoe_last_6_avg_home,qb_epa_last_6_avg_home,sacks_allowed_last_6_avg_home,interceptions_thrown_last_6_avg_home,qb_value_pre_last_6_avg_home,qb_adj_last_6_avg_home,total_rushes_last_6_avg_home,total_rush_yards_last_6_avg_home,rushing_epa_last_6_avg_home,rush_tds_last_6_avg_home,fumbles_last_6_avg_home,lost_fumbles_last_6_avg_home,ypc_last_6_avg_home,total_rushes_allowed_last_6_avg_home,total_rush_yards_allowed_last_6_avg_home,rush_tds_allowed_last_6_avg_home,total_pass_attempts_allowed_last_6_avg_home,passing_yards_allowed_last_6_avg_home,completions_allowed_last_6_avg_home,pass_tds_allowed_last_6_avg_home,avg_cpoe_allowed_last_6_avg_home,sacks_last_6_avg_home,fumbles_forced_last_6_avg_home,interceptions_last_6_avg_home,fumbles_recovered_last_6_avg_home,special_teams_epa_last_6_avg_home,passing_epa_def_last_6_avg_home,rushing_epa_def_last_6_avg_home,season_away,week_away,team_full_name_away,qb_away,total_pass_attempts_last_6_avg_away,total_passing_yards_last_6_avg_away,completions_last_6_avg_away,passing_epa_last_6_avg_away,pass_tds_last_6_avg_away,avg_cpoe_last_6_avg_away,qb_epa_last_6_avg_away,sacks_allowed_last_6_avg_away,interceptions_thrown_last_6_avg_away,qb_value_pre_last_6_avg_away,qb_adj_last_6_avg_away,total_rushes_last_6_avg_away,total_rush_yards_last_6_avg_away,rushing_epa_last_6_avg_away,rush_tds_last_6_avg_away,fumbles_last_6_avg_away,lost_fumbles_last_6_avg_away,ypc_last_6_avg_away,total_rushes_allowed_last_6_avg_away,total_rush_yards_allowed_last_6_avg_away,rush_tds_allowed_last_6_avg_away,total_pass_attempts_allowed_last_6_avg_away,passing_yards_allowed_last_6_avg_away,completions_allowed_last_6_avg_away,pass_tds_allowed_last_6_avg_away,avg_cpoe_allowed_last_6_avg_away,sacks_last_6_avg_away,fumbles_forced_last_6_avg_away,interceptions_last_6_avg_away,fumbles_recovered_last_6_avg_away,special_teams_epa_last_6_avg_away,passing_epa_def_last_6_avg_away,rushing_epa_def_last_6_avg_away
17,2021,2,CAR,NO,Carolina Panthers,New Orleans Saints,2021,2,Carolina Panthers,Sam Darnold,35.000000,279.000000,24.000000,2.872278,1.000000,10.441611,0.459338,1.000000,0.000000,104.471617,-17.899887,25.166667,109.000000,-0.185955,1.166667,1.333333,0.500000,4.391667,23.666667,117.833333,0.500000,36.500000,247.000000,23.833333,2.333333,2.851088,3.166667,1.000000,0.500000,0.666667,0.145164,1.319827,-1.626429,2021,2,New Orleans Saints,Jameis Winston,21.000000,151.000000,15.000000,16.867315,5.000000,11.732475,20.059868,0.000000,0.000000,149.475487,-15.689334,30.833333,147.166667,2.364964,1.666667,0.666667,0.333333,4.603880,23.500000,94.166667,0.833333,37.333333,236.166667,21.333333,1.500000,-3.630208,2.166667,0.833333,1.166667,0.500000,2.149643,5.695655,0.256268
18,2021,2,PHI,SF,Philadelphia Eagles,San Francisco 49ers,2021,2,Philadelphia Eagles,Jalen Hurts,34.000000,264.000000,27.000000,12.510652,3.000000,11.622833,17.783419,1.000000,0.000000,119.529623,-7.458145,29.833333,154.666667,-1.050570,1.166667,1.666667,0.500000,5.129094,26.333333,121.666667,0.666667,34.333333,286.833333,24.333333,2.166667,6.444036,2.666667,0.500000,0.833333,0.500000,0.704835,-2.151557,1.786563,2021,2,San Francisco 49ers,Jimmy Garoppolo,25.000000,319.000000,18.000000,19.006603,2.000000,7.501111,9.618484,1.000000,0.000000,127.748569,-4.082609,27.333333,131.666667,-2.160570,0.833333,1.166667,1.166667,4.766033,24.833333,106.166667,0.833333,41.833333,245.666667,26.000000,1.833333,-2.675135,2.166667,0.500000,0.500000,0.166667,-5.679718,3.551542,-0.372279
19,2021,2,MIA,BUF,Miami Dolphins,Buffalo Bills,2021,2,Miami Dolphins,Tua Tagovailoa,27.000000,202.000000,16.000000,1.342094,1.000000,-4.562320,2.526210,2.000000,1.000000,128.278939,-12.355889,26.166667,119.833333,1.812055,1.166667,0.333333,0.333333,4.365366,22.66666

In [50]:
final_nfl.to_csv('../data/predictive_stats_df.csv')